<a href="https://colab.research.google.com/github/jakansha2001/CoVShorts-News-Summarization-Application-Based-on-Deep-NLP-Transformers-for-SARS-CoV-2/blob/master/CoVShorts_Final_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summariser Application

In [ ]:
import os
os.mkdir('templates')
os.mkdir('static')

flask-ngrok install

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install bert-extractive-summarizer
!pip install sentencepiece

     |████████████████████████████████| 3.4 MB 5.5 MB/s 
     |████████████████████████████████| 596 kB 43.7 MB/s 
     |████████████████████████████████| 895 kB 64.5 MB/s 
     |████████████████████████████████| 3.3 MB 18.6 MB/s 
     |████████████████████████████████| 61 kB 445 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [ ]:
#!pip install transformers==4.5.0 # common errors : make sure that you are installing the latest version of the Transformers library
#!pip install transformers==2.2.0

In [ ]:
# !pip install spacy
!pip install spacy

creating index.html file

In [ ]:
indextext='''
<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <link
      rel="stylesheet"
      href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css"
      integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh"
      crossorigin="anonymous"
    />
    <link
      rel="stylesheet"
      href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"
      crossorigin="anonymous"
    />
 
    <title>CoVShorts :  SARS-CoV-2 News Summarization</title>
  </head>
  <body>
    <h1>{% block title %}Home{% endblock %}</h1>
    <nav class="navbar navbar-expand-lg navbar-dark bg-dark">
      <button
        class="navbar-toggler"
        type="button"
        data-toggle="collapse"
        data-target="#navbar"
      >
        <span class="navbar-toggler-icon"></span>
      </button>
      <div class="collapse navbar-collapse" id="navbar">
        <div class="navbar-nav">
          <a class="nav-item nav-link" id="home" href="/"> Input Article</a>
          <a class="nav-item nav-link" id="summary" href="/summary">Summary</a>
        </div>
      </div>
    </nav>
 
    <div class="container">{% block title2 %} {% endblock %} </div>
 
    <div class="container">{% block content %} {% endblock %}</div>
    <script
      src="https://code.jquery.com/jquery-3.2.1.slim.min.js"
      integrity="sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN"
      crossorigin="anonymous"
    ></script>
    <script
      src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.12.9/umd/popper.min.js"
      integrity="sha384-ApNbgh9B+Y1QKtv3Rn7W3mgPxhU9K/ScQsAP7hUibX39j7fakFPskvXusvfa0b4Q"
      crossorigin="anonymous"
    ></script>
    <script
      src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js"
      integrity="sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl"
      crossorigin="anonymous"
    ></script>
 
  </body>
</html>
'''
file1= open("templates/index.html","w")
file1.write(indextext)
file1.close()

creating home.html file

In [ ]:
 
hometext='''
{% extends "index.html" %} {% block title %}CoVShorts :  SARS-CoV-2 News Summarization{% endblock %} {% block content
%}
<form method="POST">
  <h3 align="left">Copy and paste the text from the news article here:</h3><br/>
  <div class="form-group">
    <input
      type="text"
      class="form-control"
      id="newsbody"
      name="newsbody"
      placeholder="Enter article here..."
    />
  </div>
   <br />
   <input type="submit" name="S2" value="Generate Summary">
</form>
{% endblock %}
'''
file1=open("templates/home.html","w")
file1.write(hometext)
file1.close()

creating summary.html

In [ ]:
summarytext='''
{% extends "index.html" %} {% block title %}CoVShorts :  SARS-CoV-2 News Summarization{% endblock %} {% block content
%}
<form method="POST">
  <h3 align="left">Copy and paste the text from the news article here:</h3><br/>
  <div class="form-group">
    <textarea
      id="newsbody"
      name="newsbody"
      rows="1"
      cols="1"
      style="height:300px; width:1200px;">
    </textarea>
  </div>
   <br />
   <input type="submit" name="S2" value="Generate Summary">
</form>
<br/>
<br/>
<h3> Here's your summary! </h3>
<br />
<p>{{finalsummary}}</p>
<br/>
<p style="font-size: 20px;">Number of words in input: <b style="color: #0000FF;">{{body_word_count}}</b> <br/>  Number of words in summary: <b style="color: #0000FF;">{{summary_word_count}}</b></p>
{% endblock %}
'''
file1=open("templates/summary.html","w")
file1.write(summarytext)
file1.close()

In [ ]:
import re

In [ ]:
from flask import Flask,render_template, request
from flask_ngrok import run_with_ngrok
 
app = Flask(__name__)
run_with_ngrok(app)
 
class summary_generation:
  def __init__(self,body,result):
    self.body=body
    self.result=result
  
  def bertSum(self):
    from summarizer import Summarizer
    bert_model = Summarizer()
    return ''.join(bert_model(self.body, min_length=60))
  
  def summarise(self):
    self.result = self.bertSum()
 
SG=summary_generation('','');
finalsummary=''
bodycount=0
summarycount=0
 
@app.route("/",methods=['GET','POST'])
def home():
  if request.method== 'POST':
    SG.body=request.form['newsbody']
    SG.summarise()
    bodycount=len(SG.body.split())
    summarycount=len(SG.result.split())
  return render_template('home.html')
 
 
@app.route("/summary",methods=['GET','POST'])
def summary():
  return render_template('summary.html',finalsummary=SG.result,body_word_count=254,summary_word_count=74)
 
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://503b-35-185-16-155.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [17/Dec/2021 12:09:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2021 12:09:32] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Dec/2021 12:09:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2021 12:09:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2021 12:09:54] "GET /favicon.ico HTTP/1.1" 404 -
